In [49]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA

In [50]:
raw_train = pd.read_csv("./input/train.csv")
h_trs = pd.read_csv("./input/historical_transactions.csv")
merchants = pd.read_csv("./input/merchants.csv")

In [51]:
pca = PCA(n_components=1)
merchants["numerical_pca"] = pca.fit_transform(merchants[["numerical_1", "numerical_2"]].values)

In [52]:
df_merchants_numerical = merchants[["merchant_id", "numerical_pca"]]

In [53]:
h_trs = pd.merge(h_trs, df_merchants_numerical, on="merchant_id", how="left")

In [54]:
h_trs["exp_numerical_pca"] = np.exp(h_trs.numerical_pca)
h_trs["relu_numerical_pca"] = np.where(h_trs.numerical_pca>0, h_trs.numerical_pca, 0)

In [55]:
numerical_h_trs = h_trs[["card_id", "numerical_pca", "exp_numerical_pca", "relu_numerical_pca"]]

In [56]:
agg_func = {
        'numerical_pca': ['sum', 'mean', 'max', 'min', 'std'],
        'exp_numerical_pca': ['sum', 'mean', 'max', 'min', 'std'],
        'relu_numerical_pca': ['sum', 'mean', 'max', 'std']
        }

In [57]:
agg_history = numerical_h_trs.groupby(['card_id']).agg(agg_func)
agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
agg_history.reset_index(inplace=True)

In [58]:
agg_history.to_csv("./input/merchants_numerical_features.csv", index=False)